In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import r2_score

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv('./dataset/cleaned.csv')

In [2]:
# getting dummie variables

df_dummies = df.filter(['city', 'state', 'event_type', 'flight_condition','flight_operation', 'sightseeing', 'second_pilot'], axis=1)
df_dummies = pd.get_dummies(df_dummies, columns=['city', 'state', 'event_type', 'flight_condition','flight_operation', 'sightseeing', 'second_pilot'], drop_first=True)

In [3]:
# set any deaths or injuries to 1

for i in range(len(df['deaths_and_injuries'])):
    if df['deaths_and_injuries'][i] > 0:
        df['deaths_and_injuries'][i] = 1

<ipython-input-3-3ff8f6f3afb8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['deaths_and_injuries'][i] = 1


In [4]:
# setting up X and y

X = df_dummies.values
y = df['deaths_and_injuries'].astype(int).values

In [5]:
# train test split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train[0].shape[0]

5166

===

### Model 1

In [7]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [2048],
    'l3': [1024],
    'l4': [512],
    'l5': [256],
    'drop': [.75, .8, 85]
}

gs = GridSearchCV(nn, param_grid=params, cv=5)

gs.fit(X_train, y_train)

print(gs.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs.best_params_

<ipython-input-7-3e053e861464>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
Epoch 1/50
34/34 [==============================] - 14s 341ms/step - loss: 0.7134 - acc: 0.5239
Epoch 2/50
34/34 [==============================] - 12s 349ms/step - loss: 0.6998 - acc: 0.5292
Epoch 3/50
34/34 [==============================] - 12s 343ms/step - loss: 0.6928 - acc: 0.5294
Epoch 4/50
34/34 [==============================] - 12s 342ms/step - loss: 0.6906 - acc: 0.5375
Epoch 5/50
34/34 [==============================] - 11s 326ms/step - loss: 0.6873 - acc: 0.5414
Epoch 6/50
34/34 [==============================] - 11s 327ms/step - loss: 0.6837 - acc: 0.5489
Epoch 7/50
34/34 [==============================] - 11s 332ms/step - loss: 0.6755 - acc: 0.5503
Epoch 8/50
34/34 [==============================] - 12s 345ms/step - loss: 0.6676 - acc: 0.5612
Epoch 9/50
34/34 [==============================] - 11s 333ms/step - loss: 0.6456 - acc: 0.6120
Epoch 10/50
34/34 [==============================] - 11s 324ms/step - los

34/34 [==============================] - 12s 361ms/step - loss: 0.2304 - acc: 0.8786
Epoch 34/50
34/34 [==============================] - 13s 371ms/step - loss: 0.2334 - acc: 0.8791
Epoch 35/50
34/34 [==============================] - 12s 366ms/step - loss: 0.2342 - acc: 0.8740
Epoch 36/50
34/34 [==============================] - 13s 379ms/step - loss: 0.2286 - acc: 0.8780
Epoch 37/50
34/34 [==============================] - 12s 354ms/step - loss: 0.2239 - acc: 0.8818
Epoch 38/50
34/34 [==============================] - 12s 367ms/step - loss: 0.2188 - acc: 0.8850
Epoch 39/50
34/34 [==============================] - 12s 367ms/step - loss: 0.2173 - acc: 0.8819
Epoch 40/50
34/34 [==============================] - 12s 357ms/step - loss: 0.2198 - acc: 0.8834
Epoch 41/50
34/34 [==============================] - 12s 361ms/step - loss: 0.2195 - acc: 0.8797
Epoch 42/50
34/34 [==============================] - 12s 355ms/step - loss: 0.2190 - acc: 0.8799
Epoch 43/50
34/34 [=======================

34/34 [==============================] - 12s 352ms/step - loss: 0.4468 - acc: 0.8072
Epoch 15/50
34/34 [==============================] - 11s 327ms/step - loss: 0.4220 - acc: 0.8171
Epoch 16/50
34/34 [==============================] - 11s 332ms/step - loss: 0.3919 - acc: 0.8250
Epoch 17/50
34/34 [==============================] - 11s 330ms/step - loss: 0.3704 - acc: 0.8345
Epoch 18/50
34/34 [==============================] - 11s 331ms/step - loss: 0.3475 - acc: 0.8420
Epoch 19/50
34/34 [==============================] - 11s 333ms/step - loss: 0.3242 - acc: 0.8531
Epoch 20/50
34/34 [==============================] - 11s 317ms/step - loss: 0.3118 - acc: 0.8502
Epoch 21/50
34/34 [==============================] - 8s 225ms/step - loss: 0.3068 - acc: 0.8552
Epoch 22/50
34/34 [==============================] - 10s 306ms/step - loss: 0.2964 - acc: 0.8563
Epoch 23/50
34/34 [==============================] - 11s 332ms/step - loss: 0.2909 - acc: 0.8613
Epoch 24/50
34/34 [========================

34/34 [==============================] - 12s 357ms/step - loss: 0.2113 - acc: 0.8839
Epoch 48/50
34/34 [==============================] - 13s 368ms/step - loss: 0.2072 - acc: 0.8885
Epoch 49/50
34/34 [==============================] - 12s 365ms/step - loss: 0.2093 - acc: 0.8879
Epoch 50/50
9/9 [==============================] - 2s 66ms/step - loss: 1.9210 - acc: 0.5452
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
Epoch 1/50
34/34 [==============================] - 15s 364ms/step - loss: 0.7386 - acc: 0.5150
Epoch 2/50
34/34 [==============================] - 12s 362ms/step - loss: 0.7069 - acc: 0.5169
Epoch 3/50
34/34 [==============================] - 11s 316ms/step - loss: 0.6971 - acc: 0.5248
Epoch 4/50
34/34 [==============================] - 7s 207ms/step - loss: 0.6921 - acc: 0.5375
Epoch 5/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6933 - acc: 0.5232
Epoch 6/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6900 - acc:

...

C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-7-3e053e861464>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256


C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-7-3e053e861464>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

Epoch 1/50
42/42 [==============================] - 9s 200ms/step - loss: 0.7342 - acc: 0.5161
Epoch 2/50
42/42 [==============================] - 9s 202ms/step - loss: 0.7025 - acc: 0.5170
Epoch 3/50
42/42 [==============================] - 8s 201ms/step - loss: 0.6955 - acc: 0.5268
Epoch 4/50
42/42 [==============================] - 9s 209ms/step - loss: 0.6930 - acc: 0.5328
Epoch 5/50
42/42 [==============================] - 9s 222ms/step - loss: 0.6912 - acc: 0.5342
Epoch 6/50
42/42 [==============================] - 8s 200ms/step - loss: 0.6902 - acc: 0.5296
Epoch 7/50
42/42 [==============================] - 9s 203ms/step - loss: 0.6862 - acc: 0.5406
Epoch 8/50
42/42 [==============================] - 8s 199ms/step - loss: 0.6837 - acc: 0.5438
Epoch 9/50
42/42 [==============================] - 9s 204ms/step - loss: 0.6810 - acc: 0.5432
Epoch 10/50
42/42 [==============================] - 9s 204ms/step - loss: 0.6760 - acc: 0.5601
Epoch 11/50
42/42 [==============================

{'drop': 0.8,
 'epochs': 50,
 'l1': 4096,
 'l2': 2048,
 'l3': 1024,
 'l4': 512,
 'l5': 256}

In [8]:
r2_score(y_test, gs.predict(X_test))

14/14 [==============================] - 1s 50ms/step


-0.2997154581696093

===

### Model 2

In [9]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [1024],
    'l3': [256],
    'l4': [64],
    'l5': [16],
    'drop': [.75, .8, 85]
}

gs2 = GridSearchCV(nn, param_grid=params, cv=5)

gs2.fit(X_train, y_train)

print(gs2.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs2.best_params_

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


<ipython-input-9-beb3aba4a063>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 6s 147ms/step - loss: 0.7253 - acc: 0.5132
Epoch 2/50
34/34 [==============================] - 5s 148ms/step - loss: 0.7015 - acc: 0.5166
Epoch 3/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6916 - acc: 0.5354
Epoch 4/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6930 - acc: 0.5388
Epoch 5/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6916 - acc: 0.5450
Epoch 6/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6916 - acc: 0.5417
Epoch 7/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6904 - acc: 0.5466
Epoch 8/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6901 - acc: 0.5447
Epoch 9/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6895 - acc: 0.5439
Epoch 10/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6884 - acc: 0.5483
Epoch 11/50
34/34 [==============================

34/34 [==============================] - 5s 148ms/step - loss: 0.6423 - acc: 0.6182
Epoch 36/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6286 - acc: 0.6303
Epoch 37/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6265 - acc: 0.6420
Epoch 38/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6101 - acc: 0.6516
Epoch 39/50
34/34 [==============================] - 5s 146ms/step - loss: 0.5998 - acc: 0.6716
Epoch 40/50
34/34 [==============================] - 5s 150ms/step - loss: 0.5954 - acc: 0.6831
Epoch 41/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5972 - acc: 0.6773
Epoch 42/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5834 - acc: 0.6832
Epoch 43/50
34/34 [==============================] - 5s 149ms/step - loss: 0.5683 - acc: 0.7074
Epoch 44/50
34/34 [==============================] - 5s 150ms/step - loss: 0.5571 - acc: 0.7142
Epoch 45/50
34/34 [==============================] -

34/34 [==============================] - 5s 151ms/step - loss: 0.6821 - acc: 0.5437
Epoch 18/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6814 - acc: 0.5449
Epoch 19/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6781 - acc: 0.5443
Epoch 20/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6738 - acc: 0.5437
Epoch 21/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6669 - acc: 0.5452
Epoch 22/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6583 - acc: 0.5443
Epoch 23/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6494 - acc: 0.5659
Epoch 24/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6430 - acc: 0.6324
Epoch 25/50
34/34 [==============================] - 5s 144ms/step - loss: 0.6289 - acc: 0.6593
Epoch 26/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6234 - acc: 0.6593
Epoch 27/50
34/34 [==============================] -

...

C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-9-beb3aba4a063>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Henry\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-9-beb3aba4a063>", line 8, in model_func
    model.add(Dropout(drop))
  File "C:\Users\Henry\anaconda3\lib\site-packages\keras\layers\core\dropout.py", line 80, in __init__
    raise ValueError(f'Invalid value {rate} received for '
ValueError: Invalid value 85 received for `rate`, expected a value between 0 and 1.

  warnings.warn("Estimator fit failed. The score on this train-test

Epoch 1/50
42/42 [==============================] - 7s 149ms/step - loss: 0.7375 - acc: 0.5135
Epoch 2/50
42/42 [==============================] - 6s 150ms/step - loss: 0.7017 - acc: 0.4986
Epoch 3/50
42/42 [==============================] - 6s 150ms/step - loss: 0.6957 - acc: 0.5280
Epoch 4/50
42/42 [==============================] - 6s 154ms/step - loss: 0.6935 - acc: 0.5466
Epoch 5/50
42/42 [==============================] - 6s 152ms/step - loss: 0.6929 - acc: 0.5418
Epoch 6/50
42/42 [==============================] - 6s 149ms/step - loss: 0.6936 - acc: 0.5431
Epoch 7/50
42/42 [==============================] - 6s 151ms/step - loss: 0.6915 - acc: 0.5458
Epoch 8/50
42/42 [==============================] - 6s 150ms/step - loss: 0.6895 - acc: 0.5454
Epoch 9/50
42/42 [==============================] - 6s 149ms/step - loss: 0.6897 - acc: 0.5442
Epoch 10/50
42/42 [==============================] - 6s 152ms/step - loss: 0.6898 - acc: 0.5457
Epoch 11/50
42/42 [==============================

{'drop': 0.8,
 'epochs': 50,
 'l1': 4096,
 'l2': 1024,
 'l3': 256,
 'l4': 64,
 'l5': 16}

In [10]:
r2_score(y_test, gs2.predict(X_test))

14/14 [==============================] - 1s 40ms/step


-0.08263015689444164

===

### Model 3

In [11]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, l6=32, l7=32, l8=32, l9=32, l10=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5,
         'layer_6', l6, 'layer_7', l7, 'layer_8', l8, 'layer_9', l9, 'layer_10', l10)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    
    
    model.add(Dense(l6, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l7, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l8, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l9, activation='relu'))
    model.add(Dropout(drop)) 

    model.add(Dense(l10, activation='relu'))
    model.add(Dropout(drop)) 
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [2048],
    'l3': [1024],
    'l4': [512],
    'l5': [256],
    'l6': [128],
    'l7': [64],
    'l8': [32],
    'l9': [16],
    'l10': [8],
    'drop': [.5, .6, .7]
}

gs3 = GridSearchCV(nn, param_grid=params, cv=5)

gs3.fit(X_train, y_train)

print(gs3.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs3.best_params_

layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256 layer_6 128 layer_7 64 layer_8 32 layer_9 16 layer_10 8


<ipython-input-11-3bbc520ff0b0>:44: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 8s 203ms/step - loss: 0.7019 - acc: 0.5182
Epoch 2/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6917 - acc: 0.5370
Epoch 3/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6893 - acc: 0.5431
Epoch 4/50
34/34 [==============================] - 7s 205ms/step - loss: 0.6897 - acc: 0.5419
Epoch 5/50
34/34 [==============================] - 7s 207ms/step - loss: 0.6875 - acc: 0.5477
Epoch 6/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6875 - acc: 0.5441
Epoch 7/50
34/34 [==============================] - 7s 206ms/step - loss: 0.6821 - acc: 0.5442 1s - loss: 0.6817 - acc: 
Epoch 8/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6796 - acc: 0.5449
Epoch 9/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6677 - acc: 0.5433
Epoch 10/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6542 - acc: 0.5438
Epoch 11/50
34/34 [====

34/34 [==============================] - 7s 204ms/step - loss: 0.3804 - acc: 0.8459
Epoch 34/50
34/34 [==============================] - 7s 197ms/step - loss: 0.3626 - acc: 0.8558
Epoch 35/50
34/34 [==============================] - 7s 208ms/step - loss: 0.3441 - acc: 0.8565
Epoch 36/50
34/34 [==============================] - 7s 203ms/step - loss: 0.3441 - acc: 0.8579
Epoch 37/50
34/34 [==============================] - 7s 204ms/step - loss: 0.3446 - acc: 0.8574
Epoch 38/50
34/34 [==============================] - 7s 199ms/step - loss: 0.3279 - acc: 0.8650
Epoch 39/50
34/34 [==============================] - 7s 200ms/step - loss: 0.3304 - acc: 0.8650
Epoch 40/50
34/34 [==============================] - 7s 204ms/step - loss: 0.3241 - acc: 0.8594
Epoch 41/50
34/34 [==============================] - 7s 199ms/step - loss: 0.3296 - acc: 0.8616
Epoch 42/50
34/34 [==============================] - 7s 200ms/step - loss: 0.3279 - acc: 0.8626
Epoch 43/50
34/34 [==============================] -

34/34 [==============================] - 7s 199ms/step - loss: 0.5026 - acc: 0.7718
Epoch 14/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4846 - acc: 0.7788
Epoch 15/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4740 - acc: 0.7838
Epoch 16/50
34/34 [==============================] - 7s 205ms/step - loss: 0.4603 - acc: 0.7906
Epoch 17/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4534 - acc: 0.7960
Epoch 18/50
34/34 [==============================] - 7s 203ms/step - loss: 0.4547 - acc: 0.8011
Epoch 19/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4478 - acc: 0.7956
Epoch 20/50
34/34 [==============================] - 7s 202ms/step - loss: 0.4356 - acc: 0.8023
Epoch 21/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4405 - acc: 0.7930
Epoch 22/50
34/34 [==============================] - 7s 198ms/step - loss: 0.4430 - acc: 0.7962
Epoch 23/50
34/34 [==============================] -

...

34/34 [==============================] - 7s 203ms/step - loss: 0.5005 - acc: 0.7461
Epoch 38/50
34/34 [==============================] - 7s 200ms/step - loss: 0.5006 - acc: 0.7481
Epoch 39/50
34/34 [==============================] - 7s 201ms/step - loss: 0.5030 - acc: 0.7479
Epoch 40/50
34/34 [==============================] - 7s 203ms/step - loss: 0.4896 - acc: 0.7571
Epoch 41/50
34/34 [==============================] - 7s 205ms/step - loss: 0.4796 - acc: 0.7641
Epoch 42/50
34/34 [==============================] - 7s 202ms/step - loss: 0.4786 - acc: 0.7667
Epoch 43/50
34/34 [==============================] - 7s 204ms/step - loss: 0.4715 - acc: 0.7723 4s - loss
Epoch 44/50
34/34 [==============================] - 7s 201ms/step - loss: 0.4682 - acc: 0.7753
Epoch 45/50
34/34 [==============================] - 7s 199ms/step - loss: 0.4601 - acc: 0.7774
Epoch 46/50
34/34 [==============================] - 7s 206ms/step - loss: 0.4596 - acc: 0.7823
Epoch 47/50
34/34 [=======================

34/34 [==============================] - 7s 200ms/step - loss: 0.6899 - acc: 0.5470 
Epoch 18/50
34/34 [==============================] - 7s 203ms/step - loss: 0.6889 - acc: 0.5470 
Epoch 19/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6891 - acc: 0.5459
Epoch 20/50
34/34 [==============================] - 7s 203ms/step - loss: 0.6889 - acc: 0.5467
Epoch 21/50
34/34 [==============================] - 7s 203ms/step - loss: 0.6888 - acc: 0.5466
Epoch 22/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6887 - acc: 0.5465
Epoch 23/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6890 - acc: 0.5468
Epoch 24/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6887 - acc: 0.5468
Epoch 25/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6910 - acc: 0.5446
Epoch 26/50
34/34 [==============================] - 7s 205ms/step - loss: 0.6887 - acc: 0.5461
Epoch 27/50
34/34 [==============================]

9/9 [==============================] - 1s 55ms/step - loss: 0.6905 - acc: 0.5382
layer_1 4096 layer_2 2048 layer_3 1024 layer_4 512 layer_5 256 layer_6 128 layer_7 64 layer_8 32 layer_9 16 layer_10 8
Epoch 1/50
34/34 [==============================] - 8s 201ms/step - loss: 0.8124 - acc: 0.5112
Epoch 2/50
34/34 [==============================] - 7s 204ms/step - loss: 0.7063 - acc: 0.5244
Epoch 3/50
34/34 [==============================] - 7s 198ms/step - loss: 0.6971 - acc: 0.5301
Epoch 4/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6930 - acc: 0.5433
Epoch 5/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6923 - acc: 0.5375
Epoch 6/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6923 - acc: 0.5396
Epoch 7/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6913 - acc: 0.5429
Epoch 8/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6909 - acc: 0.5440
Epoch 9/50
34/34 [======================

34/34 [==============================] - 7s 204ms/step - loss: 0.6893 - acc: 0.5451
Epoch 32/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6895 - acc: 0.5458
Epoch 33/50
34/34 [==============================] - 7s 201ms/step - loss: 0.6888 - acc: 0.5461
Epoch 34/50
34/34 [==============================] - 7s 204ms/step - loss: 0.6888 - acc: 0.5458
Epoch 35/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6875 - acc: 0.5466
Epoch 36/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6893 - acc: 0.5451 3s - loss: 
Epoch 37/50
34/34 [==============================] - 7s 202ms/step - loss: 0.6885 - acc: 0.5452
Epoch 38/50
34/34 [==============================] - 7s 200ms/step - loss: 0.6875 - acc: 0.5447 4s - 
Epoch 39/50
34/34 [==============================] - 7s 207ms/step - loss: 0.6851 - acc: 0.5450
Epoch 40/50
34/34 [==============================] - 7s 199ms/step - loss: 0.6846 - acc: 0.5461
Epoch 41/50
34/34 [===============

{'drop': 0.7,
 'epochs': 50,
 'l1': 4096,
 'l10': 8,
 'l2': 2048,
 'l3': 1024,
 'l4': 512,
 'l5': 256,
 'l6': 128,
 'l7': 64,
 'l8': 32,
 'l9': 16}

In [12]:
r2_score(y_test, gs3.predict(X_test))

14/14 [==============================] - 1s 53ms/step


-0.002619670208006575

===

### Model 4

In [13]:
def model_func(l1=32, l2=32, l3=32, l4=32, l5=32, drop=.5):
    
    print('layer_1', l1, 'layer_2', l2, 'layer_3', l3, 'layer_4', l4, 'layer_5', l5)
    
    model = Sequential()
    
    model.add(Dense(l1, activation='relu', input_shape=(5166,)))
    model.add(Dropout(drop))
    
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(drop))
 
    model.add(Dense(l3, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l4, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(l5, activation='relu'))
    model.add(Dropout(drop))    

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)

params = {
    'epochs': [50],
    'l1': [4096],
    'l2': [1024],
    'l3': [256],
    'l4': [64],
    'l5': [16],
    'drop': [.65, .7, .75, .8, .85]
}

gs4 = GridSearchCV(nn, param_grid=params, cv=5)

gs4.fit(X_train, y_train)

print(gs4.best_score_) # <- negative mean squared error, ie the higher (less negative) the better
gs4.best_params_

layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16


<ipython-input-13-c73e0408d965>:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn=model_func, batch_size=256, verbose=1)


Epoch 1/50
34/34 [==============================] - 6s 146ms/step - loss: 0.7002 - acc: 0.5163
Epoch 2/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6929 - acc: 0.5258
Epoch 3/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6907 - acc: 0.5336
Epoch 4/50
34/34 [==============================] - 5s 149ms/step - loss: 0.6908 - acc: 0.5339
Epoch 5/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6882 - acc: 0.5393
Epoch 6/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6879 - acc: 0.5405
Epoch 7/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6847 - acc: 0.5429
Epoch 8/50
34/34 [==============================] - 5s 152ms/step - loss: 0.6822 - acc: 0.5477
Epoch 9/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6805 - acc: 0.5456
Epoch 10/50
34/34 [==============================] - 5s 150ms/step - loss: 0.6761 - acc: 0.5454
Epoch 11/50
34/34 [==============================

34/34 [==============================] - 5s 150ms/step - loss: 0.2538 - acc: 0.8586
Epoch 35/50
34/34 [==============================] - 5s 151ms/step - loss: 0.2451 - acc: 0.8615
Epoch 36/50
34/34 [==============================] - 5s 150ms/step - loss: 0.2453 - acc: 0.8650
Epoch 37/50
34/34 [==============================] - 5s 150ms/step - loss: 0.2418 - acc: 0.8626
Epoch 38/50
34/34 [==============================] - 5s 154ms/step - loss: 0.2327 - acc: 0.8591
Epoch 39/50
34/34 [==============================] - 5s 147ms/step - loss: 0.2289 - acc: 0.8591
Epoch 40/50
34/34 [==============================] - 5s 150ms/step - loss: 0.2359 - acc: 0.8655
Epoch 41/50
34/34 [==============================] - 5s 151ms/step - loss: 0.2338 - acc: 0.8613
Epoch 42/50
34/34 [==============================] - 5s 150ms/step - loss: 0.2259 - acc: 0.8619
Epoch 43/50
34/34 [==============================] - 5s 150ms/step - loss: 0.2319 - acc: 0.8593
Epoch 44/50
34/34 [==============================] -

34/34 [==============================] - 5s 149ms/step - loss: 0.5035 - acc: 0.7775
Epoch 17/50
34/34 [==============================] - 5s 150ms/step - loss: 0.4597 - acc: 0.8022
Epoch 18/50
34/34 [==============================] - 5s 147ms/step - loss: 0.4359 - acc: 0.8105
Epoch 19/50
34/34 [==============================] - 5s 147ms/step - loss: 0.3945 - acc: 0.8196
Epoch 20/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3807 - acc: 0.8254
Epoch 21/50
34/34 [==============================] - 5s 149ms/step - loss: 0.3573 - acc: 0.8301
Epoch 22/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3547 - acc: 0.8360
Epoch 23/50
34/34 [==============================] - 5s 149ms/step - loss: 0.3408 - acc: 0.8375
Epoch 24/50
34/34 [==============================] - 5s 148ms/step - loss: 0.3241 - acc: 0.8396
Epoch 25/50
34/34 [==============================] - 5s 150ms/step - loss: 0.3126 - acc: 0.8485
Epoch 26/50
34/34 [==============================] -

...

34/34 [==============================] - 5s 148ms/step - loss: 0.6806 - acc: 0.5456
Epoch 45/50
34/34 [==============================] - 5s 144ms/step - loss: 0.6813 - acc: 0.5459
Epoch 46/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6785 - acc: 0.5458
Epoch 47/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6801 - acc: 0.5452
Epoch 48/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6731 - acc: 0.5468
Epoch 49/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6708 - acc: 0.5460
Epoch 50/50
9/9 [==============================] - 1s 42ms/step - loss: 0.6827 - acc: 0.5517
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_5 16
Epoch 1/50
34/34 [==============================] - 5s 145ms/step - loss: 0.8546 - acc: 0.4953
Epoch 2/50
34/34 [==============================] - 5s 147ms/step - loss: 0.7319 - acc: 0.5190
Epoch 3/50
34/34 [==============================] - 5s 145ms/step - loss: 0.7130 - acc: 0.532

34/34 [==============================] - 5s 147ms/step - loss: 0.6892 - acc: 0.5446
Epoch 27/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6903 - acc: 0.5443
Epoch 28/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6888 - acc: 0.5466
Epoch 29/50
34/34 [==============================] - 5s 151ms/step - loss: 0.6906 - acc: 0.5454
Epoch 30/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6888 - acc: 0.5461
Epoch 31/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6898 - acc: 0.5458
Epoch 32/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6896 - acc: 0.5454
Epoch 33/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6891 - acc: 0.5456
Epoch 34/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6908 - acc: 0.5442
Epoch 35/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6900 - acc: 0.5439
Epoch 36/50
34/34 [==============================] -

34/34 [==============================] - 5s 146ms/step - loss: 0.6922 - acc: 0.5402
Epoch 9/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6931 - acc: 0.5366
Epoch 10/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6914 - acc: 0.5452
Epoch 11/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6935 - acc: 0.5364
Epoch 12/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6937 - acc: 0.5365
Epoch 13/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6907 - acc: 0.5416
Epoch 14/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6911 - acc: 0.5407
Epoch 15/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6909 - acc: 0.5433
Epoch 16/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6911 - acc: 0.5422
Epoch 17/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6907 - acc: 0.5451
Epoch 18/50
34/34 [==============================] - 

34/34 [==============================] - 5s 146ms/step - loss: 0.6893 - acc: 0.5447
Epoch 42/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6897 - acc: 0.5458
Epoch 43/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6897 - acc: 0.5459
Epoch 44/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6895 - acc: 0.5460
Epoch 45/50
34/34 [==============================] - 5s 147ms/step - loss: 0.6895 - acc: 0.5463
Epoch 46/50
34/34 [==============================] - 5s 148ms/step - loss: 0.6898 - acc: 0.5447
Epoch 47/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6886 - acc: 0.5467
Epoch 48/50
34/34 [==============================] - 5s 145ms/step - loss: 0.6894 - acc: 0.5453
Epoch 49/50
34/34 [==============================] - 5s 146ms/step - loss: 0.6894 - acc: 0.5460
Epoch 50/50
9/9 [==============================] - 1s 37ms/step - loss: 0.6878 - acc: 0.5517
layer_1 4096 layer_2 1024 layer_3 256 layer_4 64 layer_

{'drop': 0.85,
 'epochs': 50,
 'l1': 4096,
 'l2': 1024,
 'l3': 256,
 'l4': 64,
 'l5': 16}

In [14]:
r2_score(y_test, gs4.predict(X_test))

14/14 [==============================] - 1s 44ms/step


-0.0005036162201546013